In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from mapping import ResourceMapper
from graph import *
from sparql import SPAQLConverter

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')
rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)
sc = SPAQLConverter(config)

# 1. Preprocessing

# 2-1. Query Partitioning

In [4]:
user_query = '용감한 형제가 작곡한 노래의 장르는?'
# user_query = '뱅뱅뱅 부른 음악그룹의 소속사는?'
# user_query = 'This Love 부른 음악가의 소속 음악 그룹은?'
query_terms = rm.tokenize_query(user_query)
query_terms

['용감한', '형제', '작곡', '노래', '장르']

In [5]:
query_patitions = rm.partition_query(query_terms)
query_patitions

[['용감한 형제 작곡 노래 장르'],
 ['용감한', '형제 작곡 노래 장르'],
 ['용감한 형제', '작곡 노래 장르'],
 ['용감한 형제 작곡', '노래 장르'],
 ['용감한 형제 작곡 노래', '장르'],
 ['용감한', '형제', '작곡 노래 장르'],
 ['용감한', '형제 작곡', '노래 장르'],
 ['용감한', '형제 작곡 노래', '장르'],
 ['용감한 형제', '작곡', '노래 장르'],
 ['용감한 형제', '작곡 노래', '장르'],
 ['용감한 형제 작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래 장르'],
 ['용감한', '형제', '작곡 노래', '장르'],
 ['용감한', '형제 작곡', '노래', '장르'],
 ['용감한 형제', '작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래', '장르']]

# 2-2. Partition Scoring

In [6]:
p_star, score_list = rm.score_partition(query_patitions)
p_star

['용감한 형제', '작곡', '노래', '장르']

# 2-3. Generation of resource combination

In [7]:
resource_combinations = rm.combinate_resource(p_star)
resource_combinations

[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:Genre'),
 ('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:hasGenre')]

# 3. Conceptual graph generation

In [8]:
conceptual_graph = cgg.generate_conceptual_graph(resource_combinations)
conceptual_graph

[[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'Any P', 'skmo:Genre')],
 [('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'skmo:hasGenre', 'owl:Thing')]]

# 4. Query graph generation

In [9]:
query_graph = qgg.generate_query_graph(conceptual_graph)
query_graph

[(0.7517241379310344,
  [('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]),
 (0.7517241379310344,
  [('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre'),
   ('skmo:Genre', 'rdfs:subClassOf', 'owl:Thing')])]

# 5. SPARQL Conversion

In [10]:
knows_query = sc.generate_query(query_graph)
print(knows_query)

result = sc.excute_query(knows_query)
for r in result:
    print(r['target'])

SELECT DISTINCT ?target 
WHERE
{
    ?x0 skmo:isComposedBy skmo:용감한_형제; skmo:hasGenre ?target
}
http://www.sktelecom.com/skmo/k-pop
http://www.sktelecom.com/skmo/R&B
http://www.sktelecom.com/skmo/hip_hop
